In [1]:
import pandas as pd

In [4]:
df_effective = pd.read_excel('data/dallas_apartment_data_rent_sf.xlsx',sheet_name = 'Supply')
df_effective

,MSA/Submarket,Total Units,2023 Deliveries,2024 Deliveries (YTD),2024 Net Absorption (YTD)
0,Dallas/Fort Worth Overall,831427,31670,22194,14138
1,Bachman Lake/Webb Chapel,10162,0,0,-116
2,Carrollton North,13934,0,600,-311
3,Carrollton South,10793,379,0,303
4,Central Arlington,12463,0,0,-93
5,Central Fort Worth,23854,1531,1360,485
6,Denton,17346,1724,1712,1036
7,Downtown/The Cedars/Deep Ellum,15750,481,411,378
8,Duncanville/Desoto/Lancaster,11322,196,65,222
9,East Dallas,12578,0,0,-159


In [49]:
df_me = pd.read_csv('data/m_and_e_stacked.csv')

In [50]:
df_me.drop('Unnamed: 0',axis = 1,inplace = True)

In [51]:
df_me.head()

,MSA/Submarket,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31
0,Dallas/Fort Worth Overall_market,1.80,1.79,1.78,1.77,1.77,1.76,1.76,1.76,1.76,1.77,1.77,1.77
1,Bachman Lake/Webb Chapel_market,1.63,1.65,1.66,1.67,1.66,1.69,1.68,1.68,1.68,1.69,1.69,1.69
2,Carrollton North_market,1.84,1.84,1.83,1.83,1.82,1.83,1.83,1.83,1.82,1.82,1.82,1.81
3,Carrollton South_market,1.84,1.84,1.83,1.83,1.82,1.83,1.84,1.82,1.84,1.83,1.83,1.84
4,Central Arlington_market,1.52,1.54,1.52,1.52,1.53,1.54,1.54,1.55,1.54,1.54,1.54,1.54


In [52]:
df_mar = df_me[df_me['MSA/Submarket'].str.contains('_market')].reset_index()
df_eff = df_me[df_me['MSA/Submarket'].str.contains('_effective')].reset_index()

In [53]:
def find_means(df,method):
    df_gd = df.drop('MSA/Submarket',axis=1)
    mean_groupby = df_gd.agg(method)
    mean_groupby['MSA/Submarket'] = f"{method} value"
    df.loc[len(df)+1]=mean_groupby
    return df

In [54]:
df_mar_edited = find_means(df_mar,'mean')
df_mar_edited_ = find_means(df_mar_edited,'std')
df_eff_edited = find_means(df_eff,'mean')
df_eff_edited_ = find_means(df_eff_edited,'std')

In [55]:
df_mar_edited.index = df_mar_edited['MSA/Submarket']
df_mar_edited=df_mar_edited.drop('MSA/Submarket',axis=1)
df_eff_edited.index = df_eff_edited['MSA/Submarket']
df_eff_edited=df_eff_edited.drop('MSA/Submarket',axis=1)

In [56]:
df_mar_edited.to_csv('data/edited_market.csv')
df_eff_edited.to_csv('data/edited_effective.csv')

In [57]:
df_me.index = df_me['MSA/Submarket']
df_me.drop('MSA/Submarket',axis=1,inplace=True)

In [58]:
market = 'Dallas/Fort Worth Overall'
#mark_df = df_mar_edited[df_mar_edited['MSA/Submarket'].str.contains(market)]


In [59]:
def get_differences(df,df_type,market_selection):
    df.loc['value_difference'] = df.loc['mean value'] - df.loc[f'{market_selection}_{df_type}']
    return df

In [60]:
df_mar_edited = get_differences(df_mar_edited,'market',market)
df_eff_edited = get_differences(df_eff_edited,'effective',market)

In [61]:
df_full = pd.concat([df_mar_edited,df_eff_edited])
df_full.loc['market_effective_difference'] = df_full.loc[f'{market}_market'] - df_full.loc[f'{market}_effective']
df_full

,index,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31
MSA/Submarket,,,,,,,,,,,,,
Dallas/Fort Worth Overall_market,0.000000,1.800000,1.790000,1.780000,1.770000,1.770000,1.760000,1.760000,1.760000,1.760000,1.770000,1.770000,1.770000
Bachman Lake/Webb Chapel_market,1.000000,1.630000,1.650000,1.660000,1.670000,1.660000,1.690000,1.680000,1.680000,1.680000,1.690000,1.690000,1.690000
Carrollton North_market,2.000000,1.840000,1.840000,1.830000,1.830000,1.820000,1.830000,1.830000,1.830000,1.820000,1.820000,1.820000,1.810000
Carrollton South_market,3.000000,1.840000,1.840000,1.830000,1.830000,1.820000,1.830000,1.840000,1.820000,1.840000,1.830000,1.830000,1.840000
Central Arlington_market,4.000000,1.520000,1.540000,1.520000,1.520000,1.530000,1.540000,1.540000,1.550000,1.540000,1.540000,1.540000,1.540000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
White Rock East/Buckner_effective,87.000000,1.550000,1.540000,1.540000,1.550000,1.550000,1.540000,1.540000,1.520000,1.530000,1.540000,1.540000,1.550000
mean value,65.500000,1.754773,1.748864,1.736364,1.723636,1.716818,1.711136,1.707500,1.710682,1.712727,1.714318,1.719318,1.718864
std value,12.698425,0.209810,0.207257,0.204663,0.199067,0.197263,0.193099,0.193651,0.193431,0.193982,0.195697,0.203128,0.206334


In [62]:
#momentum the current minus the value of certain period of time ago (I will do month to month)
#The metric will be meaned and stddev 
#Use the RSI to determine the strength
#100 - [100/(1+(average gain/average loss))]
#can add the current gain and the current loss for smoothing
#can do the current rsi and the average momentum
#Percentage Increase = [(Final Value - Initial Value)/Initial Value] × 100.
#can either do the average momentum or the final momentum

In [63]:
df_len = len(df_me.columns)
for i,j in enumerate(df_me.columns):
    if i+1 < df_len:
        print(i+1)
        col_first = df_me.columns[i]
        print(col_first)
        col_second = df_me.columns[i+1]
        print(col_second)
        val_col = f'vel_{col_first}_and_{col_second}'
        print(val_col)
        df_me[val_col] = df_me[col_second] - df_me[col_first]
    

1
2023-08-31
2023-09-30
vel_2023-08-31_and_2023-09-30
2
2023-09-30
2023-10-31
vel_2023-09-30_and_2023-10-31
3
2023-10-31
2023-11-30
vel_2023-10-31_and_2023-11-30
4
2023-11-30
2023-12-31
vel_2023-11-30_and_2023-12-31
5
2023-12-31
2024-01-31
vel_2023-12-31_and_2024-01-31
6
2024-01-31
2024-02-29
vel_2024-01-31_and_2024-02-29
7
2024-02-29
2024-03-31
vel_2024-02-29_and_2024-03-31
8
2024-03-31
2024-04-30
vel_2024-03-31_and_2024-04-30
9
2024-04-30
2024-05-31
vel_2024-04-30_and_2024-05-31
10
2024-05-31
2024-06-30
vel_2024-05-31_and_2024-06-30
11
2024-06-30
2024-07-31
vel_2024-06-30_and_2024-07-31


In [64]:
df_me

,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,...,vel_2023-09-30_and_2023-10-31,vel_2023-10-31_and_2023-11-30,vel_2023-11-30_and_2023-12-31,vel_2023-12-31_and_2024-01-31,vel_2024-01-31_and_2024-02-29,vel_2024-02-29_and_2024-03-31,vel_2024-03-31_and_2024-04-30,vel_2024-04-30_and_2024-05-31,vel_2024-05-31_and_2024-06-30,vel_2024-06-30_and_2024-07-31
MSA/Submarket,,,,,,,,,,,,,,,,,,,,,
Dallas/Fort Worth Overall_market,1.80,1.79,1.78,1.77,1.77,1.76,1.76,1.76,1.76,1.77,...,-0.01,-0.01,0.00,-0.01,0.00,0.00,0.00,0.01,0.00,0.00
Bachman Lake/Webb Chapel_market,1.63,1.65,1.66,1.67,1.66,1.69,1.68,1.68,1.68,1.69,...,0.01,0.01,-0.01,0.03,-0.01,0.00,0.00,0.01,0.00,0.00
Carrollton North_market,1.84,1.84,1.83,1.83,1.82,1.83,1.83,1.83,1.82,1.82,...,-0.01,0.00,-0.01,0.01,0.00,0.00,-0.01,0.00,0.00,-0.01
Carrollton South_market,1.84,1.84,1.83,1.83,1.82,1.83,1.84,1.82,1.84,1.83,...,-0.01,0.00,-0.01,0.01,0.01,-0.02,0.02,-0.01,0.00,0.01
Central Arlington_market,1.52,1.54,1.52,1.52,1.53,1.54,1.54,1.55,1.54,1.54,...,-0.02,0.00,0.01,0.01,0.00,0.01,-0.01,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Southwest Dallas/Redbird_effective,1.43,1.45,1.42,1.42,1.43,1.42,1.42,1.43,1.43,1.43,...,-0.03,0.00,0.01,-0.01,0.00,0.01,0.00,0.00,0.00,0.01
Uptown/Downtown/Park Cities_effective,2.62,2.62,2.60,2.58,2.56,2.52,2.53,2.52,2.53,2.57,...,-0.02,-0.02,-0.02,-0.04,0.01,-0.01,0.01,0.04,0.03,0.02
West Fort Worth_effective,1.44,1.44,1.43,1.43,1.43,1.40,1.41,1.41,1.40,1.43,...,-0.01,0.00,0.00,-0.03,0.01,0.00,-0.01,0.03,0.01,-0.02


In [65]:
df_me.columns

Index(['2023-08-31', '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31',
       '2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30', '2024-05-31',
       '2024-06-30', '2024-07-31', 'vel_2023-08-31_and_2023-09-30',
       'vel_2023-09-30_and_2023-10-31', 'vel_2023-10-31_and_2023-11-30',
       'vel_2023-11-30_and_2023-12-31', 'vel_2023-12-31_and_2024-01-31',
       'vel_2024-01-31_and_2024-02-29', 'vel_2024-02-29_and_2024-03-31',
       'vel_2024-03-31_and_2024-04-30', 'vel_2024-04-30_and_2024-05-31',
       'vel_2024-05-31_and_2024-06-30', 'vel_2024-06-30_and_2024-07-31'],
      dtype='object')

In [66]:
percent_gains = []
for i,j in enumerate(df_me.columns):
    if i+1 < df_len:
        col_first = df_me.columns[i]
        col_second = df_me.columns[i+1]
        val_col = f'percents_{col_first}_and_{col_second}'
        print(val_col)
        df_me[val_col] = (df_me[col_second] - df_me[col_first])/df_me[col_first]

percents_2023-08-31_and_2023-09-30
percents_2023-09-30_and_2023-10-31
percents_2023-10-31_and_2023-11-30
percents_2023-11-30_and_2023-12-31
percents_2023-12-31_and_2024-01-31
percents_2024-01-31_and_2024-02-29
percents_2024-02-29_and_2024-03-31
percents_2024-03-31_and_2024-04-30
percents_2024-04-30_and_2024-05-31
percents_2024-05-31_and_2024-06-30
percents_2024-06-30_and_2024-07-31


In [67]:
df_me

,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,...,percents_2023-09-30_and_2023-10-31,percents_2023-10-31_and_2023-11-30,percents_2023-11-30_and_2023-12-31,percents_2023-12-31_and_2024-01-31,percents_2024-01-31_and_2024-02-29,percents_2024-02-29_and_2024-03-31,percents_2024-03-31_and_2024-04-30,percents_2024-04-30_and_2024-05-31,percents_2024-05-31_and_2024-06-30,percents_2024-06-30_and_2024-07-31
MSA/Submarket,,,,,,,,,,,,,,,,,,,,,
Dallas/Fort Worth Overall_market,1.80,1.79,1.78,1.77,1.77,1.76,1.76,1.76,1.76,1.77,...,-0.005587,-0.005618,0.000000,-0.005650,0.000000,0.000000,0.000000,0.005682,0.000000,0.000000
Bachman Lake/Webb Chapel_market,1.63,1.65,1.66,1.67,1.66,1.69,1.68,1.68,1.68,1.69,...,0.006061,0.006024,-0.005988,0.018072,-0.005917,0.000000,0.000000,0.005952,0.000000,0.000000
Carrollton North_market,1.84,1.84,1.83,1.83,1.82,1.83,1.83,1.83,1.82,1.82,...,-0.005435,0.000000,-0.005464,0.005495,0.000000,0.000000,-0.005464,0.000000,0.000000,-0.005495
Carrollton South_market,1.84,1.84,1.83,1.83,1.82,1.83,1.84,1.82,1.84,1.83,...,-0.005435,0.000000,-0.005464,0.005495,0.005464,-0.010870,0.010989,-0.005435,0.000000,0.005464
Central Arlington_market,1.52,1.54,1.52,1.52,1.53,1.54,1.54,1.55,1.54,1.54,...,-0.012987,0.000000,0.006579,0.006536,0.000000,0.006494,-0.006452,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Southwest Dallas/Redbird_effective,1.43,1.45,1.42,1.42,1.43,1.42,1.42,1.43,1.43,1.43,...,-0.020690,0.000000,0.007042,-0.006993,0.000000,0.007042,0.000000,0.000000,0.000000,0.006993
Uptown/Downtown/Park Cities_effective,2.62,2.62,2.60,2.58,2.56,2.52,2.53,2.52,2.53,2.57,...,-0.007634,-0.007692,-0.007752,-0.015625,0.003968,-0.003953,0.003968,0.015810,0.011673,0.007692
West Fort Worth_effective,1.44,1.44,1.43,1.43,1.43,1.40,1.41,1.41,1.40,1.43,...,-0.006944,0.000000,0.000000,-0.020979,0.007143,0.000000,-0.007092,0.021429,0.006993,-0.013889


In [68]:
df_me.columns

Index(['2023-08-31', '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31',
       '2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30', '2024-05-31',
       '2024-06-30', '2024-07-31', 'vel_2023-08-31_and_2023-09-30',
       'vel_2023-09-30_and_2023-10-31', 'vel_2023-10-31_and_2023-11-30',
       'vel_2023-11-30_and_2023-12-31', 'vel_2023-12-31_and_2024-01-31',
       'vel_2024-01-31_and_2024-02-29', 'vel_2024-02-29_and_2024-03-31',
       'vel_2024-03-31_and_2024-04-30', 'vel_2024-04-30_and_2024-05-31',
       'vel_2024-05-31_and_2024-06-30', 'vel_2024-06-30_and_2024-07-31',
       'percents_2023-08-31_and_2023-09-30',
       'percents_2023-09-30_and_2023-10-31',
       'percents_2023-10-31_and_2023-11-30',
       'percents_2023-11-30_and_2023-12-31',
       'percents_2023-12-31_and_2024-01-31',
       'percents_2024-01-31_and_2024-02-29',
       'percents_2024-02-29_and_2024-03-31',
       'percents_2024-03-31_and_2024-04-30',
       'percents_2024-04-30_and_2024-05-31',
       '

In [69]:
vel_list = [i for i in df_me.columns if 'vel_' in i]
percent_list = [i for i in df_me.columns if 'percents_' in i]

In [70]:
vel_df = df_me[vel_list]
vel_df.to_csv('data/full_vel_df.csv')

In [72]:
vel_df

,vel_2023-08-31_and_2023-09-30,vel_2023-09-30_and_2023-10-31,vel_2023-10-31_and_2023-11-30,vel_2023-11-30_and_2023-12-31,vel_2023-12-31_and_2024-01-31,vel_2024-01-31_and_2024-02-29,vel_2024-02-29_and_2024-03-31,vel_2024-03-31_and_2024-04-30,vel_2024-04-30_and_2024-05-31,vel_2024-05-31_and_2024-06-30,vel_2024-06-30_and_2024-07-31
MSA/Submarket,,,,,,,,,,,
Dallas/Fort Worth Overall_market,-0.01,-0.01,-0.01,0.00,-0.01,0.00,0.00,0.00,0.01,0.00,0.00
Bachman Lake/Webb Chapel_market,0.02,0.01,0.01,-0.01,0.03,-0.01,0.00,0.00,0.01,0.00,0.00
Carrollton North_market,0.00,-0.01,0.00,-0.01,0.01,0.00,0.00,-0.01,0.00,0.00,-0.01
Carrollton South_market,0.00,-0.01,0.00,-0.01,0.01,0.01,-0.02,0.02,-0.01,0.00,0.01
Central Arlington_market,0.02,-0.02,0.00,0.01,0.01,0.00,0.01,-0.01,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
Southwest Dallas/Redbird_effective,0.02,-0.03,0.00,0.01,-0.01,0.00,0.01,0.00,0.00,0.00,0.01
Uptown/Downtown/Park Cities_effective,0.00,-0.02,-0.02,-0.02,-0.04,0.01,-0.01,0.01,0.04,0.03,0.02
West Fort Worth_effective,0.00,-0.01,0.00,0.00,-0.03,0.01,0.00,-0.01,0.03,0.01,-0.02


In [71]:
percent_df = df_me[percent_list]
percent_df.to_csv('data/full_percent_df.csv')

In [73]:
percent_df

,percents_2023-08-31_and_2023-09-30,percents_2023-09-30_and_2023-10-31,percents_2023-10-31_and_2023-11-30,percents_2023-11-30_and_2023-12-31,percents_2023-12-31_and_2024-01-31,percents_2024-01-31_and_2024-02-29,percents_2024-02-29_and_2024-03-31,percents_2024-03-31_and_2024-04-30,percents_2024-04-30_and_2024-05-31,percents_2024-05-31_and_2024-06-30,percents_2024-06-30_and_2024-07-31
MSA/Submarket,,,,,,,,,,,
Dallas/Fort Worth Overall_market,-0.005556,-0.005587,-0.005618,0.000000,-0.005650,0.000000,0.000000,0.000000,0.005682,0.000000,0.000000
Bachman Lake/Webb Chapel_market,0.012270,0.006061,0.006024,-0.005988,0.018072,-0.005917,0.000000,0.000000,0.005952,0.000000,0.000000
Carrollton North_market,0.000000,-0.005435,0.000000,-0.005464,0.005495,0.000000,0.000000,-0.005464,0.000000,0.000000,-0.005495
Carrollton South_market,0.000000,-0.005435,0.000000,-0.005464,0.005495,0.005464,-0.010870,0.010989,-0.005435,0.000000,0.005464
Central Arlington_market,0.013158,-0.012987,0.000000,0.006579,0.006536,0.000000,0.006494,-0.006452,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
Southwest Dallas/Redbird_effective,0.013986,-0.020690,0.000000,0.007042,-0.006993,0.000000,0.007042,0.000000,0.000000,0.000000,0.006993
Uptown/Downtown/Park Cities_effective,0.000000,-0.007634,-0.007692,-0.007752,-0.015625,0.003968,-0.003953,0.003968,0.015810,0.011673,0.007692
West Fort Worth_effective,0.000000,-0.006944,0.000000,0.000000,-0.020979,0.007143,0.000000,-0.007092,0.021429,0.006993,-0.013889
